## Exoplanets preprocessing

In [3]:
# imports and stuff
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [4]:
# read in the data
nasa = pd.read_csv(
    "../data/nasa_2025_02.csv",
    comment='#'
)

display(nasa)

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,...,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,0,2,...,2014-05-14,2008-01,2014-05-14,2.0,1.0,2.0,0.0,0.0,0.0,0.0
1,2,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,0,2,...,2014-07-23,2011-08,2014-07-23,2.0,1.0,2.0,0.0,0.0,0.0,0.0
2,3,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,1,2,...,2023-09-19,2023-08,2023-09-19,2.0,1.0,2.0,0.0,0.0,0.0,0.0
3,4,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,...,2018-09-04,2017-03,2018-09-06,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,5,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,0,1,...,2018-04-25,2009-10,2014-05-14,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38090,38091,ups And d,ups And,d,HD 9826,HIP 7513,TIC 189576919,Gaia DR2 348020448377061376,1,2,...,2019-01-28,2011-01,2019-01-31,5.0,1.0,10.0,1.0,0.0,0.0,0.0
38091,38092,ups Leo b,ups Leo,b,NaN,NaN,TIC 49430557,Gaia DR2 3794167001116433152,1,1,...,2022-01-10,2021-12,2022-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38092,38093,xi Aql b,xi Aql,b,HD 188310,HIP 97938,TIC 375464367,Gaia DR2 4298361114750843904,0,1,...,2014-07-23,2011-08,2014-07-23,1.0,1.0,1.0,0.0,0.0,0.0,0.0
38093,38094,xi Aql b,xi Aql,b,HD 188310,HIP 97938,TIC 375464367,Gaia DR2 4298361114750843904,0,1,...,2014-05-14,2008-06,2014-05-14,1.0,1.0,1.0,0.0,0.0,0.0,0.0


### Removing controversial planets

In [5]:
print(nasa.shape)
nasa = nasa[nasa['pl_controv_flag'] != 1]
print(nasa.shape)

(38095, 288)
(38020, 288)


## Merging Observations of the same planet into one planet
For now, if there are different values in rows to be merged we will just take their average.\
We could also debate on rather taking the median, doing a majority vote or something similar

In [6]:
print("Amount of distinct planets: ", len(pd.unique(nasa['pl_name'])))

Amount of distinct planets:  5803


Remove unwanted columns

In [7]:
print(nasa.columns)

# nasa = nasa.filter(like="pl_") # another idea of how it could be done...

relevant_columns = [
    'pl_name', 'hostname', 'pl_orbper', 'pl_orbsmax', 'pl_orbeccen',
    'pl_rade', 'pl_bmasse', 'pl_insol', 'pl_eqt',
    'st_teff', 'st_rad', 'st_mass', 'st_met', 'pl_masse',
]

nasa = nasa[relevant_columns]
print(nasa.columns)

Index(['rowid', 'pl_name', 'hostname', 'pl_letter', 'hd_name', 'hip_name',
       'tic_id', 'gaia_id', 'default_flag', 'sy_snum',
       ...
       'rowupdate', 'pl_pubdate', 'releasedate', 'pl_nnotes', 'st_nphot',
       'st_nrvc', 'st_nspec', 'pl_nespec', 'pl_ntranspec', 'pl_ndispec'],
      dtype='object', length=288)
Index(['pl_name', 'hostname', 'pl_orbper', 'pl_orbsmax', 'pl_orbeccen',
       'pl_rade', 'pl_bmasse', 'pl_insol', 'pl_eqt', 'st_teff', 'st_rad',
       'st_mass', 'st_met', 'pl_masse'],
      dtype='object')


In [8]:
# Here we can simply list how aggregating should be done for each column
def aggregate_observations(col):
    if pd.api.types.is_numeric_dtype(col):
        return col.median()
    else:
        return col.iloc[0] # Takes the first value


nasa = nasa.groupby('pl_name').agg(aggregate_observations).reset_index()
display(nasa)

,pl_name,hostname,pl_orbper,pl_orbsmax,pl_orbeccen,pl_rade,pl_bmasse,pl_insol,pl_eqt,st_teff,st_rad,st_mass,st_met,pl_masse
0,11 Com b,11 Com,324.620000,1.210000,0.2345,NaN,5434.700000,NaN,NaN,4808.00,16.380,2.600,-0.3050,NaN
1,11 UMi b,11 UMi,516.219985,1.530000,0.0800,NaN,3432.400000,NaN,NaN,4276.50,26.935,1.800,0.0100,NaN
2,14 And b,14 And,186.300000,0.775000,0.0000,NaN,1131.151300,NaN,NaN,4850.50,11.275,1.780,-0.2250,NaN
3,14 Her b,14 Her,1766.410000,2.810000,0.3682,NaN,1523.958885,NaN,NaN,5297.47,1.000,0.910,0.4050,2559.47216
4,16 Cyg B b,16 Cyg B,799.475000,1.678000,0.6821,NaN,533.942200,NaN,NaN,5747.00,1.130,1.000,0.0600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5798,ups And b,ups And,4.617111,0.059140,0.0200,NaN,218.531000,NaN,NaN,6131.14,1.630,1.290,0.1115,NaN
5799,ups And c,ups And,241.240500,0.828387,0.2495,NaN,629.437500,NaN,NaN,6105.51,1.630,1.295,0.1220,4443.24113
5800,ups And d,ups And,1279.803500,2.523500,0.2815,NaN,1307.867185,NaN,NaN,6105.51,1.630,1.295,0.1220,3257.74117
5801,ups Leo b,ups Leo,385.200000,1.180000,0.3200,NaN,162.092490,NaN,NaN,4836.00,11.220,1.480,-0.2000,NaN


## Export the processed data

In [9]:
nasa.to_csv('../data/nasa_aggregated.csv')